In [ ]:
from mmengine.utils import get_git_hash
from mmengine.utils.dl_utils import collect_env as collect_base_env

import mmdet


def collect_env():
    """Collect the information of the running environments."""
    env_info = collect_base_env()
    env_info['MMDetection'] = f'{mmdet.__version__}+{get_git_hash()[:7]}'
    return env_info


if __name__ == '__main__':
    for name, val in collect_env().items():
        print(f'{name}: {val}')

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import torch
print(torch.cuda.is_available())  # True가 나와야 정상
print(torch.cuda.device_count())  # 사용 가능한 GPU 수 확인
print(torch.cuda.current_device())  # 현재 사용 중인 GPU 확인
print(torch.cuda.get_device_name(0))  # GPU 이름 출력

In [ ]:
from mmdet.apis import DetInferencer

# Choose to use a config
model_name = '../mmdetection/projects/CO-DETR/configs/codino/co_dino_5scale_swin_l_16xb1_16e_o365tococo.py'
# Setup a checkpoint file to load
checkpoint = './co_dino_5scale_swin_large_16e_o365tococo-614254c9.pth'

# Set the device to be used for evaluation
device = 'cuda:0'

# Initialize the DetInferencer
inferencer = DetInferencer(model_name, checkpoint, device)

# Use the detector to do inference
img = '../mmdetection/demo/demo.jpg'
result = inferencer(img, out_dir='./output')

In [ ]:
import json
import pandas as pd

# 입력 JSON 파일 경로
input_json_file = '/data/ephemeral/home/mmdetection/work_dirs/coco_detection/test.bbox.json'

# 변환할 점수 임계값 설정
score_threshold = 0.035

# 출력할 CSV 파일 경로
output_csv_file = f'./submission_{score_threshold}.csv'

# JSON 파일 읽기
with open(input_json_file, 'r') as json_file:
    outputs = json.load(json_file)

# 이미지 별로 결과를 저장하기 위한 딕셔너리
results = {}

# 변환 코드
for output in outputs:
    image_id = output['image_id']
    
    # bbox, score, category_id를 기반으로 PredictionString 작성
    bbox = output['bbox']
    score = output['score']
    category_id = output['category_id']
    
    # score 임계값 이상만 변환
    if score > score_threshold:
        prediction = f"{category_id} {score} {bbox[0]} {bbox[1]} {bbox[0]+bbox[2]} {bbox[1]+bbox[3]} "
        
        # 동일한 image_id에 대해 PredictionString을 추가
        if image_id in results:
            results[image_id] += prediction
        else:
            results[image_id] = prediction

# 결과 저장을 위한 리스트
prediction_strings = []
image_ids = []

# 딕셔너리의 결과를 CSV 형식으로 변환
for image_id, prediction_string in results.items():
    prediction_strings.append(prediction_string.strip())  # 끝 공백 제거
    image_ids.append(f"test/{str(image_id).zfill(4)}.jpg")  # 파일명 형식 맞추기

# DataFrame 생성 및 CSV 저장
submission = pd.DataFrame({
    'PredictionString': prediction_strings,
    'image_id': image_ids
})

# CSV로 저장
submission.to_csv(output_csv_file, index=False)

# 결과 확인
print(submission.head())


In [ ]:
import json
import cv2
import os

# 경로 설정
input_json_file = '/data/ephemeral/home/mmdetection/work_dirs/coco_detection/test.bbox.json'
test_image_dir = '/data/ephemeral/home/FOLD/test'
output_dir = './output'

# 출력 디렉토리 없으면 생성
os.makedirs(output_dir, exist_ok=True)

# 변환할 점수 임계값 설정
score_threshold = 0.5

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    outputs = json.load(f)

# 이미지를 한 번만 불러오기 위해 image_id를 기준으로 그룹화
image_predictions = {}

for output in outputs:
    image_id = output['image_id']
    bbox = output['bbox']  # [x1, y1, width, height]
    score = output['score']
    category_id = output['category_id']

    # score가 임계값 이상인 경우만 처리
    if score > score_threshold:
        if image_id not in image_predictions:
            image_predictions[image_id] = []
        image_predictions[image_id].append({
            'bbox': bbox,
            'score': score,
            'category_id': category_id
        })

# 이미지에 박스 그리기
for image_id, predictions in image_predictions.items():
    # 이미지 파일 경로 설정 (파일명은 "image_id.jpg" 형식으로 가정)
    image_file = os.path.join(test_image_dir, f"{str(image_id).zfill(4)}.jpg")
    output_image_file = os.path.join(output_dir, f"{str(image_id).zfill(4)}.jpg")
    
    # 이미지 열기
    image = cv2.imread(image_file)
    
    if image is None:
        print(f"이미지 파일을 찾을 수 없습니다: {image_file}")
        continue

    # 각 예측에 대해 박스를 그리기
    for prediction in predictions:
        bbox = prediction['bbox']
        score = prediction['score']
        category_id = prediction['category_id']
        
        # bbox 좌표 (x1, y1, x2, y2)
        x1, y1 = bbox[0], bbox[1]
        x2, y2 = bbox[0] + bbox[2], bbox[1] + bbox[3]
        
        # 박스 그리기 (빨간색, 두께 2)
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
        
        # 카테고리와 점수 추가 (이미지 좌상단에 텍스트로 표시)
        label = f"Class: {category_id}, Score: {score:.2f}"
        cv2.putText(image, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # 결과 이미지 저장
    cv2.imwrite(output_image_file, image)

print(f"이미지 출력 완료. 결과는 {output_dir}에 저장되었습니다.")
